In [1]:
from elon import elon_call
position = 'chr2'
path = '../../5AB.filtered.mcool'
resolution = 2000

## Run function

#### Elongated "left" loops

In [2]:
elon_call(path, resolution, position, end_bin=30, quantile_threshold=0.98, fdr_correction=0.05, qval_threshold=0.3)

INFO:Elon_MaskED:Start Elon MaskED
INFO:Elon_MaskED:Run pileup
INFO:coolpuppy:('chr2', 'chr2'): 116
INFO:coolpuppy:Total number of piled up windows: 116
INFO:Elon_MaskED:Run pileup
INFO:coolpuppy:('chr2', 'chr2'): 27
INFO:coolpuppy:Total number of piled up windows: 27
INFO:Elon_MaskED:Saving results
INFO:Elon_MaskED:Done! Running time is 8.14 sec


#### Elongated "right" loops

In [3]:
elon_call(path, resolution, position, end_bin=25, quantile_threshold=0.99, fdr_correction=0.01, qval_threshold=0.1, elong='right')

INFO:Elon_MaskED:Start Elon MaskED
INFO:Elon_MaskED:Run pileup
INFO:coolpuppy:('chr2', 'chr2'): 55
INFO:coolpuppy:Total number of piled up windows: 55
INFO:Elon_MaskED:Run pileup
INFO:coolpuppy:('chr2', 'chr2'): 9
INFO:coolpuppy:Total number of piled up windows: 9
INFO:Elon_MaskED:Saving results
INFO:Elon_MaskED:Done! Running time is 8.19 sec


#### Symmetric loops

In [8]:
elon_call(path, resolution, position, end_bin=25, quantile_threshold=0.99, fdr_correction=0.01, qval_threshold=0.1, elong=None)

INFO:Elon_MaskED:Start Elon MaskED
INFO:Elon_MaskED:Run pileup
INFO:coolpuppy:('chr2', 'chr2'): 55
INFO:coolpuppy:Total number of piled up windows: 55
INFO:Elon_MaskED:Run pileup
INFO:coolpuppy:('chr2', 'chr2'): 27
INFO:coolpuppy:Total number of piled up windows: 27
INFO:Elon_MaskED:Saving results
INFO:Elon_MaskED:Done! Running time is 8.28 sec
